# YOLOv4 Setup

## Mount with Google drive

In [ ]:
from google.colab import drive
drive.mount('/Detecting_and_mapping_of_roadside_trees_from_google_street')

## Arranging directories

In [ ]:
%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive
%mkdir Thesis
%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis
%mkdir YOLOv4
%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/YOLOv4

## Cloning darknet repo

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

## Adjusting changes & building "Makefile" 

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!make

## Download pre-trained YOLOv4 weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

# Data Augmentation 

## Prerequisite

In [ ]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

!pip install --upgrade albumentations

! pip install --upgrade albumentations

## Setting up location & folders

In [ ]:
%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/New_Dataset

%mkdir Blur_BD_740
%mkdir Blur_BD_755
%mkdir Flip_BD_740
%mkdir Flip_BD_755
%mkdir PasadenaUrbanTrees_Blur
%mkdir PasadenaUrbanTrees_Flip

## Horizontal Flip

In [ ]:
def retrieve_data(img_label_name):
  bbox = []
  im = cv2.imread(image_loc_path+img_label_name+".jpg")
  img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  fl = open(image_loc_path+img_label_name+".txt", 'r')
  data = fl.readlines()
  fl.close()
  for dt in data:
    _, x_center, y_center, width, height = map(float, dt.split(' '))
    str_bbox = x_center,y_center,width,height,class_id
    bbox.append(str_bbox)
  bboxes = np.asarray(bbox)
  return img, bboxes

In [ ]:
def flip_image(imageT):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    img_arr, bboxes = retrieve_data(imageT)
    transform = A.Compose(
        [
         A.HorizontalFlip(p=0.5)],
        bbox_params=A.BboxParams(format='yolo'))

    transformed = transform(image=img_arr, bboxes=bboxes)

    return transformed

In [ ]:
def save_flip_img_txt(imageT):
  transformed_dict = flip_image(imageT)
  # contains the image as array
  transformed_arr = transformed_dict["image"]

  # contains the resized bounding boxes
  transformed_info = np.array(list(map(list, transformed_dict["bboxes"]))).astype(float)

  img = Image.fromarray(transformed_arr, 'RGB')
  img.save(image_loc_after_flip+str(img_name_cont)+'.jpg')

  for i in range(len(transformed_info)):
    temp = str(int(transformed_info[i,4]))+' '+str(transformed_info[i,0])+' '+str(transformed_info[i,1])+' '+str(transformed_info[i,2])+' '+str(transformed_info[i,3])
    f= open(image_loc_after_flip+str(img_name_cont)+'.txt','a')
    f.write(str(temp))
    f.write("\n")
    f.close()


In [ ]:
img_name_cont = 1
class_id = 0
image_loc_after_flip = '/Thesis/MyDrive/Thesis/New_Dataset/Test_Set/Flip_Test_Set_270/'
image_loc_path = '/Thesis/MyDrive/Thesis/New_Dataset/Test_Set/Test_Set_270/'
train_valid_loc = '/content/'

for imagePath in glob.glob(image_loc_path+'*.jpg'):
    imageTitle = imagePath.split('/')[-1]
    imageT = imageTitle.split('.')[-2]
    save_flip_img_txt(str(imageT))
    img_name_cont = img_name_cont + 1

In [ ]:
img_name_cont = 1
class_id = 0
image_loc_after_flip = '/Thesis/MyDrive/Thesis/New_Dataset/Flip_BD_740/'
image_loc_path = '/Thesis/MyDrive/Thesis/New_Dataset/BD_740/'
train_valid_loc = '/content/'

for imagePath in glob.glob(image_loc_path+'*.jpg'):
    imageTitle = imagePath.split('/')[-1]
    imageT = imageTitle.split('.')[-2]
    save_flip_img_txt(str(imageT))
    img_name_cont = img_name_cont + 1

In [ ]:
img_name_cont = 1
class_id = 0
image_loc_after_flip = '/Thesis/MyDrive/Thesis/New_Dataset/Flip_BD_755/'
image_loc_path = '/Thesis/MyDrive/Thesis/New_Dataset/BD_755/'
train_valid_loc = '/content/'

for imagePath in glob.glob(image_loc_path+'*.jpg'):
    imageTitle = imagePath.split('/')[-1]
    imageT = imageTitle.split('.')[-2]
    save_flip_img_txt(str(imageT))
    img_name_cont = img_name_cont + 1

## Blur

In [ ]:
def flip_image(imageT):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    img_arr, bboxes = retrieve_data(imageT)
    # create resize transform pipeline
    transform = A.Compose(
        [
         A.Blur(blur_limit=7, always_apply=True, p=0.5)],
        bbox_params=A.BboxParams(format='yolo'))

    transformed = transform(image=img_arr, bboxes=bboxes)

    return transformed

In [ ]:
def save_flip_img_txt(imageT):
  transformed_dict = flip_image(imageT)
  # contains the image as array
  transformed_arr = transformed_dict["image"]

  # contains the resized bounding boxes
  transformed_info = np.array(list(map(list, transformed_dict["bboxes"]))).astype(float)

  img = Image.fromarray(transformed_arr, 'RGB')
  img.save(image_loc_after_flip+str(img_name_cont)+'.jpg')

  for i in range(len(transformed_info)):
    temp = str(int(transformed_info[i,4]))+' '+str(transformed_info[i,0])+' '+str(transformed_info[i,1])+' '+str(transformed_info[i,2])+' '+str(transformed_info[i,3])
    f= open(image_loc_after_flip+str(img_name_cont)+'.txt','a')
    f.write(str(temp))
    f.write("\n")
    f.close()

In [ ]:
img_name_cont = 1
class_id = 0
image_loc_after_flip = '/Thesis/MyDrive/Thesis/New_Dataset/Test_Set/Blur_Test_Set_270/'
image_loc_path = '/Thesis/MyDrive/Thesis/New_Dataset/Test_Set/Test_Set_270/'
train_valid_loc = '/content/'

for imagePath in glob.glob(image_loc_path+'*.jpg'):
    imageTitle = imagePath.split('/')[-1]
    imageT = imageTitle.split('.')[-2]
    save_flip_img_txt(str(imageT))
    img_name_cont = img_name_cont + 1

In [ ]:
img_name_cont = 1
class_id = 0
image_loc_after_flip = '/Thesis/MyDrive/Thesis/New_Dataset/Blur_BD_740/'
image_loc_path = '/Thesis/MyDrive/Thesis/New_Dataset/BD_740/'
train_valid_loc = '/content/'

for imagePath in glob.glob(image_loc_path+'*.jpg'):
    imageTitle = imagePath.split('/')[-1]
    imageT = imageTitle.split('.')[-2]
    save_flip_img_txt(str(imageT))
    img_name_cont = img_name_cont + 1

In [ ]:
img_name_cont = 1
class_id = 0
image_loc_after_flip = '/Thesis/MyDrive/Thesis/New_Dataset/Blur_BD_755/'
image_loc_path = '/Thesis/MyDrive/Thesis/New_Dataset/BD_755/'
train_valid_loc = '/content/'

for imagePath in glob.glob(image_loc_path+'*.jpg'): 
    imageTitle = imagePath.split('/')[-1]
    imageT = imageTitle.split('.')[-2]
    save_flip_img_txt(str(imageT))
    img_name_cont = img_name_cont + 1

# Arranging necessary files


## Downloading manually labeled data

In [ ]:
from pathlib import Path
from google_drive_downloader import GoogleDriveDownloader as gdd

DATA_PATH = '/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/New_Dataset.zip'
if not Path(DATA_PATH).is_file():
    gdd.download_file_from_google_drive(
        file_id='1eNcXv5PfGqbGwq5sqZVFMlP12HG1K8rd',
        dest_path=DATA_PATH,
    )

In [ ]:
%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis
!unzip \*.zip && rm *.zip

## Arranging necessary files

In [ ]:
import os
import glob

data = []

rootdir = '/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/New_Dataset'
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
      for imagePath in glob.glob(d+'/'+'*.jpg'):
        data.append(imagePath)

print(len(data))

In [ ]:
import math

train_data_list = []
valid_data_list = []
test_data_list = []

for i in range(math.ceil(len(data)*0.7)):
  train_data_list.append(data[i])

for i in range(len(data)):
  if data[i] not in train_data_list and len(valid_data_list) <= math.ceil(len(data)*0.2):
    valid_data_list.append(data[i])
  elif data[i] not in train_data_list and data[i] not in valid_data_list and len(test_data_list) <= math.ceil(len(data)*0.1):
    test_data_list.append(data[i])

print(len(data))
print(len(train_data_list))
print(len(valid_data_list))
print(len(test_data_list))


In [ ]:
obj_data = ['classes = 1',
            'train = data/train.txt',
            'valid = data/valid.txt',
            'names = data/obj.names',
            'backup = backup']

with open(r'/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/YOLOv4/darknet/data/obj.data', 'w') as fp:
    for item in obj_data:
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
obj_names_data = ['Tree']

with open(r'/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/YOLOv4/darknet/data/obj.names', 'w') as fp:
    for item in obj_names_data:
        fp.write("%s" % item)
    print('Done')

In [ ]:
with open(r'/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/YOLOv4/darknet/data/train.txt', 'w') as fp:
    for item in train_data_list:
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
with open(r'/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/YOLOv4/darknet/data/valid.txt', 'w') as fp:
    for item in valid_data_list:
        fp.write("%s\n" % item)
    print('Done')

# Training the model

In [ ]:
%cd /Thesis/MyDrive/Thesis/Yolov4/darknet
!sudo chmod +x darknet
!make
!./darknet

/Thesis/MyDrive/Thesis/Yolov4/darknet/cfg/custom-yolov4-detector.cfg (Click on path to open the file)

## From beginning

In [ ]:
!./darknet detector train data/obj.data cfg/custom-yolov4-detector.cfg yolov4.conv.137 -dont_show -map | tee output.log

## Resume training

In [ ]:
!./darknet detector train data/obj.data cfg/custom-yolov4-detector.cfg backup/custom-yolov4-detector_7100.weights -dont_show -map | tee output.log

## Checking mAP

In [ ]:
!./darknet detector map data/test.data cfg/custom-yolov4-detector.cfg backup@0.005_lr/custom-yolov4-detector_best.weights -points 0 | tee result_map.txt

## Checking mAP@0.75

In [ ]:
!./darknet detector map data/obj.data cfg/custom-yolov4-detector.cfg backup@0.005_lr/custom-yolov4-detector_best.weights -iou_thresh 0.75

# Model Result

## 0.005

In [ ]:
import os
import glob
import shutil
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_final = pd.read_csv('/Thesis/MyDrive/Thesis/CSV_result@0.005/result_of_all_weight_with_new_test_set.csv') 
df_final = df_final.drop('Unnamed: 0.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1', 1)
df_final = df_final.drop('Unnamed: 0', 1)
for i in range(0,2):
  df_final.drop(index=df_final.index[-1],axis=0,inplace=True)
# df_final.sort_values(['Iterations'], inplace=True)
df_final

In [ ]:
df_final.sort_values(['Iterations'], inplace=True)
df_final

In [ ]:
df_final.plot(x='Iterations', y=['mAP@0.5', 'F1-Score', 'Precision', 'Recall'], 
                   subplots=True, layout=(2,2), kind='line',figsize=(15, 8), grid=False, alpha = .75)

In [ ]:
df_final.plot(x='Iterations', y=['mAP@0.5'])
plt.title('Iterations Vs mAP@0.5')
plt.ylabel('mAP@0.5')
plt.xlabel('Iterations') 

In [ ]:
df_final.plot(x='Iterations', y=['F1-Score'])
plt.title('Iterations Vs Performance Metrics')
plt.ylabel('Performance Metrics')
plt.xlabel('Iterations') 

In [ ]:
df_final.plot(x='Iterations', y=['Precision'])
plt.title('Iterations Vs Performance Metrics')
plt.ylabel('Performance Metrics')
plt.xlabel('Iterations') 

In [ ]:
df_final.plot(x='Iterations', y=['Recall'])
plt.title('Iterations Vs Performance Metrics')
plt.ylabel('Performance Metrics')
plt.xlabel('Iterations') 

## 0.0025

In [ ]:
df_final = pd.read_csv('/Thesis/MyDrive/Thesis/CSV_result@0.0025/result_of_all_weight_with_new_test_set.csv') 
df_final = df_final.drop('Unnamed: 0', 1)
df_final = df_final.drop('Unnamed: 0.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1', 1)
for i in range(0,3):
  df_final.drop(index=df_final.index[-1],axis=0,inplace=True)
df_final

In [ ]:
df_final.plot(x='Iterations', y=['mAP@0.5'])
plt.title('Iterations Vs mAP@0.5')
plt.ylabel('mAP@0.5')
plt.xlabel('Iterations') 

In [ ]:
df_final.plot(x='Iterations', y=['F1-Score', 'Precision', 'Recall'])
plt.title('Iterations Vs Performance Metrics')
plt.ylabel('Performance Metrics')
plt.xlabel('Iterations') 

## 0.001

In [ ]:
df_final = pd.read_csv('/Thesis/MyDrive/Thesis/CSV_result@0.001/result_of_all_weight_with_new_test_set.csv') 
df_final = df_final.drop('Unnamed: 0', 1)
df_final = df_final.drop('Unnamed: 0.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1.1.1', 1)
for i in range(0,3):
  df_final.drop(index=df_final.index[-1],axis=0,inplace=True)
df_final

In [ ]:
df_final.plot(x='Iterations', y=['mAP@0.5'])
plt.title('Iterations Vs mAP@0.5')
plt.ylabel('mAP@0.5')
plt.xlabel('Iterations') 

In [ ]:
df_final.plot(x='Iterations', y=['F1-Score', 'Precision', 'Recall'])
plt.title('Iterations Vs Performance Metrics')
plt.ylabel('Performance Metrics')
plt.xlabel('Iterations') 

## 0.0005

In [ ]:
df_final = pd.read_csv('/Thesis/MyDrive/Thesis/CSV_result@0.001/result_of_all_weight_with_new_test_set.csv') 
df_final = df_final.drop('Unnamed: 0', 1)
df_final = df_final.drop('Unnamed: 0.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1.1', 1)
df_final = df_final.drop('Unnamed: 0.1.1.1.1.1', 1)
for i in range(0,3):
  df_final.drop(index=df_final.index[-1],axis=0,inplace=True)
df_final

In [ ]:
df_final.plot(x='Iterations', y=['mAP@0.5'])
plt.title('Iterations Vs mAP@0.5')
plt.ylabel('mAP@0.5')
plt.xlabel('Iterations') 

In [ ]:
df_final.plot(x='Iterations', y=['F1-Score', 'Precision', 'Recall'])
plt.title('Iterations Vs Performance Metrics')
plt.ylabel('Performance Metrics')
plt.xlabel('Iterations') 

# GSV implementation 

## Prerequisite

In [ ]:
!pip install gmaps
!pip install ipywidgets
!pip install widgetsnbextension
!pip install google_streetview

!pip install polyline

In [ ]:
import requests
import folium
import polyline

import glob
import cv2
from skimage import io
from PIL import Image
from google.colab.patches import cv2_imshow 
import math
from sympy import Point, Line, pi

from geographiclib.geodesic import Geodesic

import math
import numpy as np

import google_streetview.api

In [ ]:
key = "YOUR KEY"

In [ ]:
def bearing(lat1, lng1, lat2, lng2):
  return Geodesic.WGS84.Inverse(lat1, lng1, lat2, lng2)['azi1']

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
     
    # distance between latitudes and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
 
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formula
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [ ]:
def gsv_dwn(starting_lat_lng,brng):
    params = [{
      'size': '640x640', # max 640x640 pixels
      'location': starting_lat_lng,
      'heading': brng,
      'fov': '120',
      'pitch': '0',
      'key': key
    }]

    print('GSV')

    results = google_streetview.api.results(params)

    results.download_links('/content/sample_data/')

In [ ]:
def new_lat_lng(lat1, lng1, brng):

    R = 6378.1 #Radius of the Earth
    brng = math.radians(brng)
    # brng = offset_theta
    # d = d_temp #in km
    d = 0.010

    lat1 = math.radians(lat1) 
    lon1 = math.radians(lng1)

    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
        math.cos(lat1)*math.sin(d/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
                math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    print(lat2,',',lon2)
    return lat1, lon2

In [ ]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out,routes

## Taking initial inputs

In [ ]:
s_lat1, s_lng1 = 23.528633, 90.738236 # user/starting 
starting_lat_lng = str(s_lat1)+','+str(s_lng1)

d_lat2, d_lng2 = 23.529981, 90.762483 # end/destination
destination_lat_lng = str(d_lat2)+','+str(d_lng2)

test, route = get_route(s_lng1, s_lat1,d_lng2, d_lat2)
print(route)
print(test)

## Calculating intermediate coordinates

In [ ]:
new_coor_lat = [s_lat1]
new_coor_lng = [s_lng1]
new_bearing = []

for i in range(len(route)):

  if i == 0:
      hav = haversine(s_lat1, s_lng1, route[i][0], route[i][1])
      brng = bearing(s_lat1, s_lng1, route[i][0], route[i][1])
      lat1, lng1, = s_lat1, s_lng1
  else:
    hav = haversine(route[i-1][0], route[i-1][1], route[i][0], route[i][1])
    brng = bearing(route[i-1][0], route[i-1][1], route[i][0], route[i][1])
    lat1, lng1, = route[i-1][0], route[i-1][1]

  if hav > 0.010:
    while hav > 0.010:
      
      R = 6378.1 #Radius of the Earth
      new_bearing.append(brng)
      brng = math.radians(brng)
      d = 0.010

      lat1 = math.radians(lat1) 
      lon1 = math.radians(lng1)

      lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
          math.cos(lat1)*math.sin(d/R)*math.cos(brng))

      lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
                  math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

      lat2 = math.degrees(lat2)
      lon2 = math.degrees(lon2)

      new_coor_lat.append(lat2)
      new_coor_lng.append(lon2)

      hav = haversine(lat2, lon2, route[i][0], route[i][1])
      brng = bearing(lat2, lon2, route[i][0], route[i][1])

      lat1, lng1 = lat2, lon2

  else:
    new_coor_lat.append(route[i][0])
    new_coor_lng.append(route[i][1])


## Object detection on downloaded GSV images

In [ ]:
user_lat = []
user_lng = []
tree_lat = []
tree_lng = []
tree_distance = []
tree_side = []

%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/Yolov4/darknet
!sudo chmod +x darknet
!./darknet

for i in zip(new_coor_lat, new_coor_lng, new_bearing):
  inter_coors = str(i[0])+','+str(i[1])

  temp_index = new_coor_lat.index(i[0])
  brng = bearing(i[0], i[1],new_coor_lat[temp_index+1], new_coor_lng[temp_index+1])

  print('inter_coors: ',inter_coors, ' ','  Beaaring: ', i[2])
  gsv_dwn(inter_coors,brng)
  image_path = '/content/sample_data/gsv_0.jpg'
  !./darknet detector test data/obj.data cfg/custom-yolov4-detector.cfg backup/custom-yolov4-detector_best.weights {image_path} -dont_show -ext_output < data/train.txt > result.txt
  
  left_x = 0
  top_y = 0
  width = 0
  height = 0

  f = open('/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/Yolov4/darknet/result.txt', 'r')
  for line in f:
    if 'Tree' in line:
      temp = line.split('\n')
      for temp2 in temp:
        if temp2 != '':

          temp3 = temp2.split('\t')
          temp4 = temp3[1].split(' ')
          temp4 = list(filter(None, temp4))

          for i in range(len(temp4)):
            if temp4[i] == '(left_x:':
              left_x = int(temp4[i+1])
            elif temp4[i] == 'top_y:':
              top_y = int(temp4[i+1])
            elif temp4[i] == 'width:':
              width = int(temp4[i+1])
            elif temp4[i] == 'height:':
              height = temp4[i+1]
              height = int(height.replace(')',''))
              


          path = '/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/Yolov4/darknet/predictions.jpg'
          img = Image.open(path)
          image = cv2.cvtColor(io.imread(path),cv2.COLOR_BGR2RGB)
          img_width, img_height = img.size
          cv2_imshow(image)


          result = image.copy()
          x1, y1 = round(img_width/2), img_height
          x2, y2 = round(img_width/2), 0

          x3, y3 = left_x + round(width / 2), top_y + height   # left_x + round(width / 2), top_y + height

          cv2.line(result, (x1, y1), (x2, y2), (255, 255, 0, 255), 3)
          cv2.line(result, (x1, y1), (x3, y3), (255, 255, 0, 255), 3)

          
          cv2_imshow(result)




          l1 = Line((x1, y1), (x2, y2))
          l2 = Line((x1, y1), (x3, y3))

          rad = l1.angle_between(l2)
          rslt =  math.degrees(rad)
          if (l2.slope>0):
            rslt = 360 - rslt  
            

          side = ''
          if rslt <=90:
            side = 'right'
          elif rslt >= 270:
            side = 'left'


          focal_length =  5.1 #mm
          sensor_height = 8.8/1000 # 8.8 μm
          real_height = 12500 #mm 1. (12.5m) * 1000 || 2. Manually measured real height of a tree  
          image_height = img_height * 0.2645833333 #pixels
          object_height = height *0.2645833333 #pixels || It will change according to bounding box of detected tree

          distance = ( focal_length * real_height  ) / ( object_height * sensor_height )
          calculated_distance = distance/10
          d_temp = distance/1000/1000




          offset = rslt
          temp = brng + offset
          offset_theta = math.radians(temp)




          R = 6378.1 #Radius of the Earth
          print('Bearing: ',brng)
          brng = math.radians(brng)
          d = d_temp #in km

          print('inter_coors: ',inter_coors,' type: ',type(inter_coors),'  Distance: ', d_temp)

          temp_inter_coor = inter_coors.split(',')

          print('inter_coors:(lat) ',temp_inter_coor[0],'  type:  ',type(temp_inter_coor[0]),
                ' inter_coors:(lng): ',temp_inter_coor[1],'  type:  ',type(temp_inter_coor[1]))
          
          lat1 = math.radians(float(temp_inter_coor[0])) 
          lon1 = math.radians(float(temp_inter_coor[1]))

          lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
              math.cos(lat1)*math.sin(d/R)*math.cos(brng))

          lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
                      math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

          lat2 = math.degrees(lat2)
          lon2 = math.degrees(lon2)

          user_lat.append(lat1)
          user_lng.append(lng1)
          tree_lat.append(lat2)
          tree_lng.append(lon2)
          tree_distance.append(d_temp)
          tree_side.append(side)

          print('new_lat  ',lat2,'  new_lng: ',lon2)
            

## Storing new tree coordinates

In [ ]:
import pandas as pd

dataframe = {'user_lat':user_lat,
                       'user_lng':user_lng,
                       'tree_lat':tree_lat,
                       'tree_lng':tree_lng,
                       'distance':tree_distance,
                       'side':tree_side
                       }
data = pd.DataFrame.from_dict(dataframe)      
data              

## Checking free spaces for plating new trees

In [ ]:
hav_distance_list = []
possibility = []

new_tree_lat = []
new_tree_lng = []

from_lat = []
from_lng = []


for i in range(len(data)-1):

  if (data.loc[i,'tree_lat'])

  hav_dis = haversine(data.loc[i,'tree_lat'],data.loc[i,'tree_lng'], data.loc[i+1,'tree_lat'],data.loc[i+1,'tree_lng'])

  if (hav_dis*1000) >= 24:

    hav_distance_list.append(hav_dis*1000)
    possibility.append('Yes Possible')

    brng = bearing(data.loc[i,'tree_lat'],data.loc[i,'tree_lng'], data.loc[i+1,'tree_lat'],data.loc[i+1,'tree_lng'])

    R = 6378.1 #Radius of the Earth
    brng = math.radians(brng)
    d = 0.012 #in km
          
    lat1 = math.radians(float(data.loc[i,'tree_lat'])) 
    lon1 = math.radians(float(data.loc[i,'tree_lng']))

    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
              math.cos(lat1)*math.sin(d/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
                      math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    new_tree_lat.append(lat2)
    new_tree_lng.append(lon2)
    
    from_lat.append(data.loc[i,'tree_lat'])
    from_lng.append(data.loc[i,'tree_lng'])

  else:
    hav_distance_list.append(hav_dis*1000)
    possibility.append('Not Possible')

hav_distance_list.append(0)
possibility.append('N/A')

data.insert(6, 'Haversine distance', hav_distance_list, True)
data.insert(7, 'Possibility', possibility, True)
data

In [ ]:
data.to_csv('/content/sample_data/data.csv')

## Mark both existing & possible new trees coordinates

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
new_trees = []

for i in zip(new_tree_lat,new_tree_lng):
  new_trees.append(i)

In [ ]:
existing_trees = []

for i in zip(from_lat,from_lng):
  existing_trees.append(i)

In [ ]:
import gmaps

gmaps.configure(api_key="YOUR KEY")

existing_trees_layer = gmaps.symbol_layer(
existing_trees, fill_color='green', stroke_color='green', scale=5
)

new_trees_layer = gmaps.symbol_layer(
new_trees, fill_color='blue', stroke_color='blue', scale=5
)


fig = gmaps.figure()
fig.add_layer(existing_trees_layer)
fig.add_layer(new_trees_layer)
fig

# Research verification

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## Mount with Google drive

In [ ]:
from google.colab import drive
drive.mount('/Detecting_and_mapping_of_roadside_trees_from_google_street')

## Locating YOLOv4 model 

In [ ]:
%cd /Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/Yolov4/darknet
!sudo chmod +x darknet
!./darknet

## Running YOLOv4 model on captured image

In [ ]:
des = "/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/9th_Feb/T5/IMG_20220209_155048.jpg"
!./darknet detector test data/obj.data cfg/custom-yolov4-detector.cfg backup/custom-yolov4-detector_best.weights {des} -dont_show -ext_output < data/train.txt > result.txt

## Visualizing the predicted image

In [ ]:
import cv2
from skimage import io
from PIL import Image
from google.colab.patches import cv2_imshow 

path = '/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/Yolov4/darknet/predictions.jpg'
img = Image.open(path)
image = cv2.cvtColor(io.imread(path),cv2.COLOR_BGR2RGB)
width, height = img.size
print('Width: ',width,'\n' 'Height: ',height)
cv2_imshow(image)

## Retrieving Tree detection information

In [ ]:
f = open('/Detecting_and_mapping_of_roadside_trees_from_google_street/MyDrive/Thesis/Yolov4/darknet/result.txt', 'r')
for line in f:
  if 'Tree' in line:
    print(line)

## Plotting line on predicted image

In [ ]:
result = image.copy()
x1, y1 = round(width/2), height
x2, y2 = round(width/2), 0

x3, y3 = 1   +round(219      /2),115     +383   # left_x + round(width / 2), top_y + height

cv2.line(result, (x1, y1), (x2, y2), (255, 255, 0, 255), 3)
cv2.line(result, (x1, y1), (x3, y3), (255, 255, 0, 255), 3)

cv2_imshow(result)

## Finding angle in between lines 

In [ ]:
import math
from sympy import Point, Line, pi

l1 = Line((x1, y1), (x2, y2))
l2 = Line((x1, y1), (x3, y3))

rad = l1.angle_between(l2)
rslt =  math.degrees(rad)
if (l2.slope>0):
  rslt = 360 - rslt  
print('For 1st tree: ',rslt, 'Degree')

## Finding distance of detected tree using "Photogrammetry Technique"

In [ ]:
focal_length = 4 #mm
sensor_height = 0.9/1000 # 8.8 μm
real_height = 600 #mm 1. (12.5m) * 1000 || 2. Manually measured real height of a tree  
image_height = height * 0.2645833333 #pixels
object_height = 2196 *0.2645833333#pixels || It will change according to bounding box of detected tree

distance = ( focal_length * real_height  ) / ( object_height * sensor_height )
calculated_distance = distance/10
print('For 1st tree: ',distance/10, 'cm')
d_temp = distance/1000/1000

## Taking coordinates input

In [ ]:
lat1, lng1 = 23.758034,90.371294 # user/starting 
starting_lat_lng = str(lat1+','+lng1)

lat2, lng2 = 23.753572, 90.376398 # end/destination
destination_lat_lng = str(lat2+','+lng2)

## Calculating bearing between starting & destination coordinates

In [ ]:
from geographiclib.geodesic import Geodesic
brng = Geodesic.WGS84.Inverse(lat1, lng1, lat2, lng2)['azi1']
print(brng, 'Degree')

## Adding both bearing 

In [ ]:
offset = rslt
temp = brng + offset
offset_theta = math.radians(temp)
print('For 1st tree: ',temp, 'Degree')
print('For 1st tree: ',offset_theta, 'Radians')

print('\n')

## Calcuting detected tree coordinates

In [ ]:
R = 6378.1 #Radius of the Earth
brng = math.radians(brng)
# brng = offset_theta
d = d_temp #in km
# d = 0.010

lat1 = math.radians(lat1) 
lon1 = math.radians(lng1)

lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
     math.cos(lat1)*math.sin(d/R)*math.cos(brng))

lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
             math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

lat2 = math.degrees(lat2)
lon2 = math.degrees(lon2)
# lon2 = (lon2+540)%360 -180
print(lat2,',',lon2)

## Intermediate coordinates

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
     
    # distance between latitudes and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
 
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formula
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [ ]:
def bearing(lat1, lng1, lat2, lng2):
  return Geodesic.WGS84.Inverse(lat1, lng1, lat2, lng2)['azi1']

## Mark on map

In [ ]:
import gmaps

gmaps.configure(api_key="YOUR KEY")


existing_trees = (lat1, lng1)
existing_trees_layer = gmaps.symbol_layer(
existing_trees, fill_color='green', stroke_color='green', scale=5
)

new_trees = (lat2,lon2)
new_trees_layer = gmaps.symbol_layer(
new_trees, fill_color='blue', stroke_color='blue', scale=5
)
# print(type(marker_locations))
fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations)
# fig.add_layer(markers)
# fig

fig.add_layer(existing_trees_layer)
fig.add_layer(new_trees_layer)
fig

## Measuring coordinates error

In [ ]:
actual_lat, actual_lng = 23.752442,90.372702
# calculated_lat, calculated_lng = lat2 , lon2
calculated_lat, calculated_lng = 23.75243706 , 90.37270211
print(calculated_lat)
print(calculated_lng)
diff_lat = (calculated_lat - actual_lat) * (math.pi / 180.0)
diff_lng = (calculated_lng - actual_lng) * (math.pi / 180.0)

actual_lat, actual_lng = actual_lat * (math.pi / 180.0), actual_lng * (math.pi / 180.0)
error_lat = abs(diff_lat/actual_lat) 
error_lng = abs(diff_lng/actual_lng) 

print(error_lat * 100, error_lng * 100)

## Measuring distance error

In [ ]:
actual_distance = 1661
calculated_distance = 1618.424755
dif_distance = calculated_distance - actual_distance
error = abs(dif_distance)/actual_distance
print(calculated_distance)
print(error * 100)